In [1]:
import time

import numpy as np
import pandas as pd
import sklearn.datasets, sklearn.metrics, sklearn.model_selection, sklearn.tree

import subprocess, sys

In [2]:
# !cd ../src/ && make print-DEPENDS

In [38]:
!cd ../src/ && make
subprocess.call([sys.executable, "lltrees_python_import_for_debug.py"])

g++ -o build/lltrees.o -c cpp/lltrees.cpp -I/usr/include/python3.10 -I/usr/include/python3.10  -Wno-unused-result -Wsign-compare -g      -fstack-protector-strong -Wformat -Werror=format-security  -DNDEBUG -g -fwrapv -O2 -Wall -fPIC -c
In file included from /usr/include/boost/smart_ptr/detail/sp_thread_sleep.hpp:22,
                 from /usr/include/boost/smart_ptr/detail/yield_k.hpp:23,
                 from /usr/include/boost/smart_ptr/detail/spinlock_gcc_atomic.hpp:14,
                 from /usr/include/boost/smart_ptr/detail/spinlock.hpp:42,
                 from /usr/include/boost/smart_ptr/detail/spinlock_pool.hpp:25,
                 from /usr/include/boost/smart_ptr/shared_ptr.hpp:29,
                 from /usr/include/boost/shared_ptr.hpp:17,
                 from /usr/include/boost/python/converter/shared_ptr_to_python.hpp:12,
                 from /usr/include/boost/python/converter/arg_to_python.hpp:15,
                 from /usr/include/boost/python/call.hpp:15,
          

0

# make_regression

In [4]:
X, Y = sklearn.datasets.make_regression(n_samples=1000, n_features=8, n_informative=5, n_targets=1, noise=1, random_state=42)
X_train, X_test, Y_train, Y_test = sklearn.model_selection.train_test_split(X, Y, test_size=0.3, random_state=42)

In [5]:
sys.path.append('/home/alexandre/Desktop/lltrees/src/build')
import lltrees
conf ={
    'epochs' : 1,
    'learning_rate' : 1,
    'metric_name' : 'mae', # Possible metrics for the moment : mae, mse
    'lltree_max_depth' : 5,
    'lltree_min_size_split' : 2,
    'lltree_criterion' : "absolute_error",  # Possible metrics for the moment : variance, absolute_error
    'verbose' : 0, 
}
my_lltree = lltrees.lltree()
my_lltree.set_conf(conf)
my_lltree.get_conf()

start_time = time.time()
my_lltree.fit(X_train, Y_train, X_test, Y_test)
print("FIT --- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
YP = my_lltree.predict(X_test)
print("PREDICT --- %s seconds ---" % (time.time() - start_time))

print("rmse: %.2f" % np.sqrt(sklearn.metrics.mean_squared_error(Y_test,YP)))
print("mae: %.2f" % sklearn.metrics.mean_absolute_error(Y_test,YP))
print("r2: %.2f" % sklearn.metrics.r2_score(Y_test,YP))

Set : epochs
Set : learning_rate
Set : metric_name
Set : lltree_max_depth
Set : lltree_min_size_split
Set : lltree_criterion
Set : verbose
Get : lltree_max_depth : 5
Get : lltree_min_size_split : 2
Get : lltree_criterion : absolute_error
Get : epochs : 1
Get : learning_rate : 1
Get : metric_name : mae
Get : verbose : 0
FIT --- 0.05533480644226074 seconds ---
PREDICT --- 7.581710815429688e-05 seconds ---
rmse: 63.35
mae: 49.11
r2: 0.74


In [6]:
my_lltree = sklearn.tree.DecisionTreeRegressor(max_depth = 5, criterion ="absolute_error", random_state = 0)

start_time = time.time()
my_lltree.fit(X_train, Y_train)
print("FIT --- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
YP = my_lltree.predict(X_test)
print("PREDICT --- %s seconds ---" % (time.time() - start_time))

print("rmse: %.2f" % np.sqrt(sklearn.metrics.mean_squared_error(Y_test,YP)))
print("mae: %.2f" % sklearn.metrics.mean_absolute_error(Y_test,YP))
print("r2: %.2f" % sklearn.metrics.r2_score(Y_test,YP))

FIT --- 0.02120375633239746 seconds ---
PREDICT --- 0.00036978721618652344 seconds ---
rmse: 63.68
mae: 49.35
r2: 0.74


In [7]:
conf ={
    'epochs' : 50,
    'learning_rate' : 0.1,
    'metric_name' : 'mae', # Possible metrics for the moment : mae, mse
    'lltree_max_depth' : 3,
    'lltree_min_size_split' : 1,
    'lltree_criterion' : "variance",  # Possible metrics for the moment : variance, absolute_error
    'verbose' : 0,  # Possible metrics for the moment : variance, absolute_error
}
my_lltree = lltrees.lltree()
my_lltree.set_conf(conf)
my_lltree.get_conf()

start_time = time.time()
my_lltree.fit(X_train, Y_train)
print("FIT --- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
YP = my_lltree.predict(X_test)
print("PREDICT --- %s seconds ---" % (time.time() - start_time))

print("rmse: %.2f" % np.sqrt(sklearn.metrics.mean_squared_error(Y_test,YP)))
print("mae: %.2f" % sklearn.metrics.mean_absolute_error(Y_test,YP))
print("r2: %.2f" % sklearn.metrics.r2_score(Y_test,YP))

FIT --- 2.349902868270874 seconds ---
PREDICT --- 0.00034999847412109375 seconds ---
rmse: 31.42
mae: 23.36
r2: 0.94
Set : epochs
Set : learning_rate
Set : metric_name
Set : lltree_max_depth
Set : lltree_min_size_split
Set : lltree_criterion
Set : verbose
Get : lltree_max_depth : 3
Get : lltree_min_size_split : 1
Get : lltree_criterion : variance
Get : epochs : 50
Get : learning_rate : 0.1
Get : metric_name : mae
Get : verbose : 0


In [8]:
from sklearn.ensemble import GradientBoostingRegressor
my_lltree = GradientBoostingRegressor(learning_rate = 0.1, n_estimators =50, 
                                 loss = 'absolute_error', criterion = 'squared_error',
                                 random_state = 0)

start_time = time.time()
my_lltree.fit(X_train, Y_train)
print("FIT --- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
YP = my_lltree.predict(X_test)
print("PREDICT --- %s seconds ---" % (time.time() - start_time))

print("rmse: %.2f" % np.sqrt(sklearn.metrics.mean_squared_error(Y_test,YP)))
print("mae: %.2f" % sklearn.metrics.mean_absolute_error(Y_test,YP))
print("r2: %.2f" % sklearn.metrics.r2_score(Y_test,YP))

FIT --- 0.1405191421508789 seconds ---
PREDICT --- 0.000982046127319336 seconds ---
rmse: 44.56
mae: 31.80
r2: 0.87


# make_classification

In [9]:
X, Y = sklearn.datasets.make_classification(n_samples=1000, n_features=8, n_informative=5, n_classes=2, random_state=42)
X_train, X_test, Y_train, Y_test = sklearn.model_selection.train_test_split(X, Y, test_size=0.3, random_state=42)

In [10]:
# conf ={
#     'epochs' : 50,
#     'learning_rate' : 0.1,
#     'metric_name' : 'mae', # Possible metrics for the moment : mae, mse
#     'lltree_max_depth' : 3,
#     'lltree_min_size_split' : 1,
#     'lltree_criterion' : "gini",  # Possible metrics for the moment : variance, absolute_error
#     'verbose' : 1,  # Possible metrics for the moment : variance, absolute_error
# }
# my_lltree = lltrees.lltree()
# my_lltree.set_conf(conf)
# my_lltree.get_conf()

# start_time = time.time()
# my_lltree.fit(X_train, Y_train)
# print("FIT --- %s seconds ---" % (time.time() - start_time))

# start_time = time.time()
# YP = my_lltree.predict(X_test)
# print("PREDICT --- %s seconds ---" % (time.time() - start_time))

# print("accuracy_score: %.2f" % np.sqrt(sklearn.metrics.accuracy_score(Y_test,YP)))
# print("log_loss: %.2f" % sklearn.metrics.log_loss(Y_test,YP))
# print("f1_score: %.2f" % sklearn.metrics.f1_score(Y_test,YP))